In [1]:
import pandas as pd

/tmp/ipykernel_7475/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("data/reviews.csv")

In [3]:
df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG
1,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...
2,3109,295840159,2018-07-24,7415343,Laurent,"Appartement spacieux, propre,clair, et calme à..."
3,3109,553502638,2019-10-24,21159216,Anastasia,"Appartement totalement rénové, en parfait état..."
4,5396,4824,2009-06-30,19995,Sarah,Perfect location!! Nasrine was a delight and m...
...,...,...,...,...,...,...
1721447,1039842567530124081,1042702285953157587,2023-12-09,373781559,Efraim,"Petit studio, très beau et très comfortable. C..."
1721448,1039842567530124081,1043366104781947066,2023-12-10,57803453,Roman,J'ai récemment séjourné dans ce charmant studi...
1721449,1040384973527856645,1041180077560231031,2023-12-07,10445285,Matthew,Simon was a great host with excellent communic...
1721450,1040440746407092425,1043378462506110040,2023-12-10,173176451,Christine,Het verblijf bij Joffrey was geweldig! Een fij...


In [4]:
df["comments"].isna().sum()

111

In [5]:
df.shape

(1721452, 6)

In [6]:
df.dropna(axis=0, subset="comments", inplace=True)
df = df.reset_index(drop=True)

In [7]:
df.shape

(1721341, 6)

In [8]:
df["date"].min()

'2009-06-30'

In [9]:
print(df["listing_id"].unique(), df["listing_id"].nunique())

[               3109                5396                7964 ...
 1040384973527856645 1040440746407092425 1040557336371326305] 56437


In [10]:
df["listing_id"].value_counts().median()

11.0

In [11]:
df1 = pd.read_csv("data/listings.csv")

In [12]:
df1.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [13]:
df1["listing_url"]

0                       https://www.airbnb.com/rooms/3109
1                       https://www.airbnb.com/rooms/5396
2                      https://www.airbnb.com/rooms/81106
3                       https://www.airbnb.com/rooms/7397
4                       https://www.airbnb.com/rooms/7964
                               ...                       
74324    https://www.airbnb.com/rooms/1043932119757241230
74325    https://www.airbnb.com/rooms/1043947326757240041
74326    https://www.airbnb.com/rooms/1043968453109441641
74327    https://www.airbnb.com/rooms/1044178383796738008
74328    https://www.airbnb.com/rooms/1044192462569643662
Name: listing_url, Length: 74329, dtype: object

In [14]:
df1["price"]

0        $150.00
1        $146.00
2        $110.00
3        $140.00
4        $180.00
          ...   
74324    $324.00
74325     $85.00
74326    $190.00
74327    $114.00
74328    $837.00
Name: price, Length: 74329, dtype: object

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map='auto'
    )

/home/onyxia/work/bertopic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.23s/it]


In [16]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("dangvantuan/sentence-camembert-large")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.


In [17]:
[df[col].nunique() for col in ["listing_id", "id"]]

[56437, 1721341]

In [18]:
countdf = df["listing_id"].value_counts()
countdf[countdf.values == 30]

listing_id
32108562              30
797545084111711312    30
38711734              30
32252252              30
50264317              30
                      ..
936863338311264567    30
19215494              30
45841279              30
29503670              30
24596494              30
Name: count, Length: 417, dtype: int64

In [19]:
corpus = df[df["listing_id"] == 53837741].reset_index(drop=True)["comments"]

In [20]:
embeddings = embedding_model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [01:54<00:00, 114.07s/it]


In [33]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short, precise and concise label (5 words maximum) of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

prompt = system_prompt + example_prompt + main_prompt

In [34]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [35]:
import transformers

model.eval()

generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id
)


In [39]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import TextGeneration
from bertopic.vectorizers import ClassTfidfTransformer

umap_model = UMAP(n_neighbors=3, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=3, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

llm = TextGeneration(generator, prompt=prompt)
representation_model = {
    "LLM": llm,
}

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=30,
  verbose=True
)


In [40]:
# Train model
topics, probs = topic_model.fit_transform(corpus, embeddings)

2024-02-07 18:56:16,945 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-07 18:56:17,883 - BERTopic - Dimensionality - Completed ✓
2024-02-07 18:56:17,884 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-07 18:56:17,890 - BERTopic - Cluster - Completed ✓
2024-02-07 18:56:17,893 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 2/2 [06:32<00:00, 196.24s/it]
2024-02-07 19:02:50,421 - BERTopic - Representation - Completed ✓


In [41]:
# Show topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,LLM,Representative_Docs
0,0,21,0_br_paris_location_apartment,"[br, paris, location, apartment, nice, perfect...","[Paris apartment reviews: Charming, local stay...","[First of all, Diane is a very accommodating h..."
1,1,9,1_très_appartement_et_et très,"[très, appartement, et, et très, agréable, est...","[Reviews of Paris apartments: pros and cons., ...","[Très joli appartement, décoré avec bcp de goû..."
